In [22]:
with open('teste.json','r',encoding='utf-8') as f:
    f = f.read()
    lista = eval(f)


In [23]:
list_dict = []
for i in lista:
    a = eval(i)
    list_dict.append(a)

In [24]:
from re import sub

def remover_numeros(text):
    text = sub(r'\d+', '', text)
    text = sub(r'\s+', ' ',text)
    return text

def preprocess_model(lista):
    words = []
    documents = []
    for dict in lista:
        for chave, valor in dict.items():
            for j in valor:
                pattern = j.lower()
                pattern = remover_numeros(pattern)
                if pattern != '':
                    words.append(pattern)
                    documents.append((pattern, chave))
    return words, documents

In [25]:
import pickle
import random
import numpy as np
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD


def train_model(lista):
    classes = []
    for i in lista:
        classes.extend(list(i.keys()))
    words,documents = preprocess_model(lista)

    words = sorted(list(set(words)))
    classes = sorted(list(set(classes)))

    words_path = ("words.pkl")
    classes_path = ("classes.pkl")

    pickle.dump(words,open(words_path, 'wb'))
    pickle.dump(classes,open(classes_path, 'wb'))

    training = []
    output_empty = [0] * len(classes)
    for document in documents:
        bag = []
        pattern_words = document[0]
        for word in words:
            bag.append(1) if word in pattern_words else bag.append(0)
        while len(bag) < len(words):
            bag.append(0)
        output_row = list(output_empty)
        output_row[classes.index(document[1])] = 1
        training.append([bag, output_row])
    random.shuffle(training)
    training = np.array(training, dtype=object)

    x = list(training[:, 0])
    y = list(training[:, 1])

    model = Sequential()
    model.add(Dense(128, input_shape=(len(x[0]),), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(y[0]), activation='softmax'))

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])

    m = model.fit(np.array(x), np.array(y), epochs=200, batch_size=5, verbose=1)

    model_path = ("model.h5")
    model.save(model_path, m)

In [26]:
train_model(list_dict)

c:\Users\herik\OneDrive\Área de Trabalho\chatbot_hipotese\.venv\lib\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
785/785 [==============================] - 3s 3ms/step - loss: 4.4675 - accuracy: 0.1287
Epoch 2/200
785/785 [==============================] - 3s 3ms/step - loss: 3.6123 - accuracy: 0.2396
Epoch 3/200
785/785 [==============================] - 2s 3ms/step - loss: 3.1925 - accuracy: 0.2974
Epoch 4/200
785/785 [==============================] - 2s 3ms/step - loss: 2.9164 - accuracy: 0.3254
Epoch 5/200
785/785 [==============================] - 2s 3ms/step - loss: 2.7307 - accuracy: 0.3499
Epoch 6/200
785/785 [==============================] - 2s 3ms/step - loss: 2.5663 - accuracy: 0.3761
Epoch 7/200
785/785 [==============================] - 2s 3ms/step - loss: 2.4648 - accuracy: 0.3993
Epoch 8/200
785/785 [==============================] - 2s 3ms/step - loss: 2.3883 - accuracy: 0.3988
Epoch 9/200
785/785 [==============================] - 3s 3ms/step - loss: 2.2762 - accuracy: 0.4172
Epoch 10/200
785/785 [==============================] - 3s 3ms/step - loss: 2.1983 - accura

In [27]:
import pandas as pd
import numpy as np
from pickle import load
from keras.models import load_model
from random import choice

In [28]:
# retorna 0 ou 1 para cada palavra da bolsa de palavras
def bag_of_words(writing, words):
    sentence_words = writing.split()
    # cria uma matriz de N palavras
    bag = [0]*len(words)
    for setence in sentence_words:
        for i, word in enumerate(words):
            if word == setence:
                # atribui 1 no pacote de palavra se a palavra atual estiver na posição da frase
                bag[i] = 1
    return(np.array(bag))

def class_prediction(input_user):
    model = load_model('model.h5')
    words = load(open('words.pkl', 'rb'))
    classes = load(open('classes.pkl', 'rb'))
    # filtra as previsões abaixo de um limite 0.25
    prevision = bag_of_words(input_user, words)
    response_prediction = model.predict(np.array([prevision]))[0]
    results = [[index, response] for index, response in enumerate(response_prediction)]

    if "1" not in str(prevision) or len(results) == 0 :
        results = [[0, response_prediction[0]]]

    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]

In [30]:
from operator import itemgetter

response = class_prediction('falhas')
max_value = 0
classe = 'nada'
for i in response:
    for j in i:
        if 'e' not in i["probability"] or '-' not in i["probability"]:
            value = float(i["probability"])
            if value > float(max_value):
                max_value = value
                classe = i
print(classe)

# a = max(response, key=itemgetter("probability"))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc1 in position 28: invalid start byte